In [ ]:
from utils import back_translate
from utils import compute_bleu_chrf
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
file_path = "/home/mshahidul/project1/all_tran_data/dataset/Sampled_100_MedlinePlus_eng_spanish_pair.json"
with open(file_path, 'r', encoding='utf-8') as json_file:
    original_file = json.load(json_file)

In [ ]:
translated_keywords[0]['sentence']

In [ ]:
translated_keywords_path = "/home/mshahidul/project1/all_tran_data/prompt_info/translated_keywords_gpt4o_mini.json"
with open(translated_keywords_path, 'r', encoding='utf-8') as translated_keywords_file:
    translated_keywords = json.load(translated_keywords_file)

# trans=translated_keywords[0]['translated_keywords']
def chain_of_dict(trans):
    chain=[]
    for x in trans:
        ordered_languages = ["French", "Portuguese", "German", "Spanish"]
        formatted_translations = []
        data=trans[x]
        for lang in ordered_languages:
            if lang in data:
                formatted_translations.append(f"'{x}' in {lang} is '{data[lang]}'")
        # print(formatted_translations)
        chain.append(", ".join(formatted_translations))
    full_data = ". ".join(chain)
    return full_data

In [ ]:

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
                    ### Instruction:
                    {}

                    ### Input:
                    {}

                    ### Response:
                    {}"""
max_seq_length=2048
model_name = "unsloth/Qwen2.5-14B-Instruct"
model_name2=model_name.split("/")[1]
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = False,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
def inference_with_prompt(ques,trans):
    # {find_synonyms(ques)}\n
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            f"{chain_of_dict(trans)}\nUse this context to translate the input into English to Spanish:", # instruction
            ques, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    ans=tokenizer.batch_decode(outputs)
    start_marker = '### Response:\n'
    end_marker = '<|end_of_text|>'

    # Find the start and end positions
    start_index = ans[0].find(start_marker) + len(start_marker)
    end_index = ans[0].find(end_marker)
    response = ans[0][start_index:end_index].strip()
    return response.split("\n\n")[0]

In [ ]:
total_score=[]
import tqdm
for line in tqdm.tqdm(translated_keywords):
    try:
        spa_tran_prompt = inference_with_prompt(line['sentence'],line['translated_keywords'])
        
        reference_text = [line['spanish']]
        hypothesis_text = spa_tran_prompt
        score1=compute_bleu_chrf(reference_text, hypothesis_text)  


        total_score.append({
            "Original_English_sentence": line['english'],
            "Original_Spanish_sentence": line['spanish'],
            "spanish_translation_prompt": spa_tran_prompt,
            "scores_cod_prompt(bleu and chrf)": score1,
        })
    except Exception as e:
        print(f"Error processing line: {line}")
        print(e)